# Backup and restore
This demo consists basically of three parts.
1. Recovery model configuration
2. Regular backup using full and transaction log backups
3. Restoring database completely
4. Restoring database to certain point in time

First moment when we should configure recovery model is immediately after the database creation. When we begin to establish backup strategy, it's not that bad to check the recovery model once again.

## Checking and setting the recovery model

In [ ]:
use master
go

select recovery_model_desc from sys.databases where name = 'Demo'
go

alter database Demo set recovery FULL
go

## Full backup
Full backup is always first backup in the line of consequencing backups. The full backup backs up allocated part of database files and active portion of transaction log. Full backup also resets differential map of the database.

In [ ]:
backup database Demo to disk = 'f:\backup\Demo.bak'
with
init
, compression
, checksum

## Log backup

Log backup follows preceding backup executed. It backs up all transaction log records (including active part of transaction log) from last previously backed up LSN. Following _BACKUP LOG_ statement should be executed on regular basis (say, once in a hour). Let's execute following statement several times

In [ ]:
backup log Demo to disk = 'f:\backup\Demo.bak'
with
noinit
, compression
, checksum

## Inspecting a content of backup file
For inspection purposes *RESTORE* statement is used.
- RESTORE VERIFYONLY: Test for backup file readibility
- RESTORE HEADERONLY: A content of backup device
- RESTORE FILELISTONLY: Where files were placed during backup

In [ ]:
restore verifyonly from disk = 'f:\backup\Demo.bak' with checksum
restore headeronly from disk = 'f:\backup\Demo.bak'
restore filelistonly from disk = 'f:\backup\Demo.bak'

# DISASTER OCCURED!
First of all it is recommended to keep as much transactions as possible. It's technique called tail-log backup. Following statement shows it.

In [ ]:
backup log Demo to disk = 'f:\backup\DemoTailLog.bak'
with
checksum, norecovery, continue_after_error

When preceding statement is executed, the database goes to RESTORING state. From now, the process of recovery can begin.

When restoring database from more backups, we need to keep it in restoring state until all backups are restored.

The restore process begins from full backup restore.

Following statement shows whole restore of the database Demo.

In [ ]:
restore headeronly from disk = 'f:\backup\Demo.bak'
restore headeronly from disk = 'f:\backup\DemoTailLog.bak'

In [ ]:
restore database Demo from disk = 'f:\backup\Demo.bak'
with
norecovery
, replace
, file = 1

restore log Demo from disk = 'f:\backup\Demo.bak'
with
norecovery
, file = 2

restore log Demo from disk = 'f:\backup\Demo.bak'
with
norecovery
, file = 3

restore log Demo from disk = 'f:\backup\DemoTailLog.bak'
with
norecovery

Here we go! The database is recovered and up to date. Or not? Let's take a look to a state of the database.

In [ ]:
select state_desc from sys.databases where name = 'Demo'

It's very usual that when database is being restored, the DBA misses to change the _NORECOVERY_ option in last RESTORE statement to _RECOVERY_. The correction is very simple:

In [ ]:
restore log Demo from disk = 'f:\backup\DemoTailLog.bak'
with
recovery
go

select state_desc from sys.databases where name = 'Demo'

## Point-in-time recovery
In many cases disaster is a result of accidental statement. When database is set to FULL recovery model, we can restore the database to a certail point in time, or to a certain marked point. Following example shows how to restore database up to accidental marked transaction. First step is to begin "regular process".

In [ ]:
backup database Demo to disk = 'f:\backup\Demo.bak'
with
init
, compression
, checksum

In [ ]:
backup log Demo to disk = 'f:\backup\Demo.bak'
with
noinit
, compression
, checksum

## Corrupted transaction

In [ ]:
use Demo
go

execute as user = 'Dev'
select * from Sales.SalesCases

begin tran A with mark
delete Sales.SalesCases
COMMIT

select * from Sales.SalesCases
revert

checkpoint

We need to execute tail log backup as soon as possible

In [ ]:
use master
go
backup log Demo to disk = 'f:\backup\DemoTailLog.bak'
with
checksum, norecovery, continue_after_error, init

Now, the restore process follows:

In [ ]:
restore headeronly from disk = 'f:\backup\Demo.bak'
restore headeronly from disk = 'f:\backup\DemoTailLog.bak'

In [ ]:
use master
go
restore database Demo from disk = 'f:\backup\Demo.bak'
with
norecovery
, replace
, file = 1

restore log Demo from disk = 'f:\backup\Demo.bak'
with
recovery
, file = 2
, stopbeforemark = 'A'

restore log Demo from disk = 'f:\backup\DemoTailLog.bak'
with
recovery
, stopbeforemark = 'A'

In [ ]:
select state_desc  from sys.databases where name = 'Demo'

Now, let's try to see data back again...

In [ ]:
use Demo
go

execute as user = 'Dev'
select * from Sales.SalesCases
revert